# Raspagem de dados de letras de músicas
Neste notebook, acessamos a plataforma letras.mus.br para salvar todas as letras de músicas de um determinado artista em uma tabela, permitindo uma análise linguística posterior.

In [39]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

## Escolhendo o artista e requisitando o conteúdo da página HTML

In [78]:
letras = 'https://www.letras.mus.br/'
artista = 'elis-regina' #PODEMOS ESCOLHER OUTRO ARTISTA!
url = letras + artista
# condição para só incluir as composições originais do artista selecionado
original_composer = False

### Parsing: links de todas as músicas

In [79]:
soup = BeautifulSoup(content, 'html.parser')

In [80]:
soup.title

<title>Elis Regina - LETRAS.MUS.BR</title>

In [81]:
#<ul class="cnt-list"> (LISTA DE TODAS AS MÚSICAS)
#<ol class="cnt-list cnt-list--num cnt-list--col2"> (LISTA DE MÚSICAS MAIS TOCADAS)
musicas = soup.find_all("a", {"class": "song-name"})

In [82]:
musicas

[<a class="song-name" href="/elis-regina/346703/"><span>1,2,3 Balançou</span></a>,
 <a class="song-name" href="/elis-regina/101409/"><span>20 Anos Blues</span></a>,
 <a class="song-name" href="/elis-regina/424055/"><span>A Banca do Distinto</span></a>,
 <a class="song-name" href="/elis-regina/322138/"><span>A Corujinha</span></a>,
 <a class="song-name" href="/elis-regina/45660/"><span>A Dama do Apocalipse</span></a>,
 <a class="song-name" href="/elis-regina/160758/"><span>A Felicidade</span></a>,
 <a class="song-name" href="/elis-regina/424056/"><span>A Fia de Chico Brito</span></a>,
 <a class="song-name" href="/elis-regina/a-noite-tonight/"><span>A Noite (Tonight)</span></a>,
 <a class="song-name" href="/elis-regina/424057/"><span>A Noite do Meu Bem</span></a>,
 <a class="song-name" href="/elis-regina/1270019/"><span>A Palo Seco</span></a>,
 <a class="song-name" href="/elis-regina/424059/"><span>A Time For Love</span></a>,
 <a class="song-name" href="/elis-regina/1790806/"><span>A Vir

In [83]:
str_musicas = str(musicas)
#href="/belchior/959402/"
split = 'href="/' + artista + '/'
lista_musicas = str_musicas.split(split)
lista_musicas = lista_musicas[1:]
print(len(lista_musicas))
lista_musicas

386


['346703/"><span>1,2,3 Balançou</span></a>, <a class="song-name" ',
 '101409/"><span>20 Anos Blues</span></a>, <a class="song-name" ',
 '424055/"><span>A Banca do Distinto</span></a>, <a class="song-name" ',
 '322138/"><span>A Corujinha</span></a>, <a class="song-name" ',
 '45660/"><span>A Dama do Apocalipse</span></a>, <a class="song-name" ',
 '160758/"><span>A Felicidade</span></a>, <a class="song-name" ',
 '424056/"><span>A Fia de Chico Brito</span></a>, <a class="song-name" ',
 'a-noite-tonight/"><span>A Noite (Tonight)</span></a>, <a class="song-name" ',
 '424057/"><span>A Noite do Meu Bem</span></a>, <a class="song-name" ',
 '1270019/"><span>A Palo Seco</span></a>, <a class="song-name" ',
 '424059/"><span>A Time For Love</span></a>, <a class="song-name" ',
 '1790806/"><span>A Virgem de Macarena</span></a>, <a class="song-name" ',
 '1221979/"><span>A Volta</span></a>, <a class="song-name" ',
 '1148564/"><span>A Voz do Morro</span></a>, <a class="song-name" ',
 '424060/"><span>Acen

In [84]:
links_musicas = []
for link in lista_musicas:
    link = link.split('/')
    link = link[0]
    link = letras + artista + '/' + link + '/'
    links_musicas.append(link)
    print(link)

https://www.letras.mus.br/elis-regina/346703/
https://www.letras.mus.br/elis-regina/101409/
https://www.letras.mus.br/elis-regina/424055/
https://www.letras.mus.br/elis-regina/322138/
https://www.letras.mus.br/elis-regina/45660/
https://www.letras.mus.br/elis-regina/160758/
https://www.letras.mus.br/elis-regina/424056/
https://www.letras.mus.br/elis-regina/a-noite-tonight/
https://www.letras.mus.br/elis-regina/424057/
https://www.letras.mus.br/elis-regina/1270019/
https://www.letras.mus.br/elis-regina/424059/
https://www.letras.mus.br/elis-regina/1790806/
https://www.letras.mus.br/elis-regina/1221979/
https://www.letras.mus.br/elis-regina/1148564/
https://www.letras.mus.br/elis-regina/424060/
https://www.letras.mus.br/elis-regina/123928/
https://www.letras.mus.br/elis-regina/248627/
https://www.letras.mus.br/elis-regina/63477/
https://www.letras.mus.br/elis-regina/311521/
https://www.letras.mus.br/elis-regina/1251900/
https://www.letras.mus.br/elis-regina/87856/
https://www.letras.mus.

In [85]:
len(links_musicas)

386

### Lista de músicas mais tocadas

In [86]:
mais_tocadas = soup.find_all("ol", {"class": "cnt-list cnt-list--num -flex-col-2 js-song-list"})
mais_tocadas

[<ol class="cnt-list cnt-list--num -flex-col-2 js-song-list"> <li class="cnt-list-row -song" data-artist="Elis Regina" data-dns="elis-regina" data-id="45670" data-name="Como Nossos Pais" data-sharetext="Como Nossos Pais de Elis Regina" data-shareurl="https://www.letras.mus.br/elis-regina/45670/" data-url="45670"> <a href="/elis-regina/45670/" title="Como Nossos Pais"><span>Como Nossos Pais</span></a> <div class="song-options"></div> </li> <li class="cnt-list-row -song" data-artist="Elis Regina" data-dns="elis-regina" data-id="45679" data-name="O Bêbado e a Equilibrista" data-sharetext="O Bêbado e a Equilibrista de Elis Regina" data-shareurl="https://www.letras.mus.br/elis-regina/45679/" data-url="45679"> <a href="/elis-regina/45679/" title="O Bêbado e a Equilibrista"><span>O Bêbado e a Equilibrista</span></a> <div class="song-options"></div> </li> <li class="cnt-list-row -song" data-artist="Elis Regina" data-dns="elis-regina" data-id="101410" data-name="Tiro Ao Álvaro" data-sharetext="

In [87]:
str_musicas = str(mais_tocadas)
#href="/belchior/959402/"
lista_musicas_mais_tocadas = str_musicas.split(split)
lista_musicas_mais_tocadas = lista_musicas_mais_tocadas[1:]
print(len(lista_musicas_mais_tocadas))
lista_musicas_mais_tocadas

20


['45670/" title="Como Nossos Pais"><span>Como Nossos Pais</span></a> <div class="song-options"></div> </li> <li class="cnt-list-row -song" data-artist="Elis Regina" data-dns="elis-regina" data-id="45679" data-name="O Bêbado e a Equilibrista" data-sharetext="O Bêbado e a Equilibrista de Elis Regina" data-shareurl="https://www.letras.mus.br/elis-regina/45679/" data-url="45679"> <a ',
 '45679/" title="O Bêbado e a Equilibrista"><span>O Bêbado e a Equilibrista</span></a> <div class="song-options"></div> </li> <li class="cnt-list-row -song" data-artist="Elis Regina" data-dns="elis-regina" data-id="101410" data-name="Tiro Ao Álvaro" data-sharetext="Tiro Ao Álvaro de Elis Regina" data-shareurl="https://www.letras.mus.br/elis-regina/101410/" data-url="101410"> <a ',
 '101410/" title="Tiro Ao Álvaro"><span>Tiro Ao Álvaro</span></a> <div class="song-options"></div> </li> <li class="cnt-list-row -song" data-artist="Elis Regina" data-dns="elis-regina" data-id="101411" data-name="Andança" data-shar

In [88]:
links_mais_tocadas = []
for link in lista_musicas_mais_tocadas:
    link = link.split('/')
    link = link[0]
    link = letras + artista + '/' + link + '/'
    links_mais_tocadas.append(link)
    print(link)

https://www.letras.mus.br/elis-regina/45670/
https://www.letras.mus.br/elis-regina/45679/
https://www.letras.mus.br/elis-regina/101410/
https://www.letras.mus.br/elis-regina/101411/
https://www.letras.mus.br/elis-regina/45673/
https://www.letras.mus.br/elis-regina/73074/
https://www.letras.mus.br/elis-regina/45668/
https://www.letras.mus.br/elis-regina/87856/
https://www.letras.mus.br/elis-regina/63477/
https://www.letras.mus.br/elis-regina/45674/
https://www.letras.mus.br/elis-regina/123933/
https://www.letras.mus.br/elis-regina/81273/
https://www.letras.mus.br/elis-regina/91023/
https://www.letras.mus.br/elis-regina/160767/
https://www.letras.mus.br/elis-regina/91039/
https://www.letras.mus.br/elis-regina/209463/
https://www.letras.mus.br/elis-regina/75151/
https://www.letras.mus.br/elis-regina/80745/
https://www.letras.mus.br/elis-regina/87853/
https://www.letras.mus.br/elis-regina/140808/


## Função para recuperar a dados sobre uma letra de uma página web. A partir de uma url do letras.br, extrai o título da música, nome do compositor, letra e compara se é uma das mais tocadas

In [89]:
#<div class="cnt-letra p402_premium" style="font-size: 19px"> (ELEMENTO HTML QUE CONTÉM A LETRA)

def parse_lyric(url):
    #testa se é uma das músicas mais tocadas
    mais_tocada = 0
    if url in links_mais_tocadas:
        mais_tocada = 1
        
    
    try:
        req = requests.get(url)
    except requests.exceptions.RequestException as e: 
        print(e)        
    content = req.content
    soup = BeautifulSoup(content, 'html.parser')
    
    #compositor
    #<div class="letra-info_comp">
    html_comp = soup.find_all("div", {"class": "letra-info_comp"})
    html_comp = str(html_comp)
    try:
        compositor = html_comp.split('Composição: ')[1]
        compositor = compositor.split('<a')[0]
        compositor = compositor[0:-2].strip()
    except:
        compositor = 'Não identificado'
    primeiro_comp = compositor.split(' / ')[0] 
    
    
    #título da música
    #<div class="cnt-head_title">
    html_title = soup.find_all("div", {"class": "cnt-head_title"})
    html_title = str(html_title)
    titulo = html_title.split('<h1>')[1]
    titulo = titulo.split('</h1>')[0]
    
    #letra da música
    html = soup.find_all("div", {"class": "cnt-letra p402_premium"})
    html = str(html)
    estrofes = html.split('<p>')
    estrofes = estrofes[1:]

    letra = ''
    for estrofe in estrofes:
        
        #estrofe = estrofe.lower()        
        #estrofe = estrofe.replace('</p>',' ')
        #aqui optamos por manter a marcação de paragrafo, para que seja possível fazer uma análise de estrofes
        #optamos também por manter as letras maiúsculas
        #esses detalhes podem facilmente ser modificados em uma análise posterior
        
        estrofe = estrofe.replace('<br/>',' ')
        estrofe = estrofe.replace('</br>',' ')
        estrofe = estrofe.replace('<br>',' ')
        estrofe = estrofe.replace('</div>]','')
        letra += estrofe

    letra = letra.strip()
    return titulo, compositor, primeiro_comp, mais_tocada, letra

In [90]:
#teste
parse_lyric('https://www.letras.mus.br/elis-regina/45670/')

('Como Nossos Pais',
 'Antonio Carlos Belchior',
 'Antonio Carlos Belchior',
 1,
 'Não quero lhe falar Meu grande amor De coisas que aprendi Nos discos Quero lhe contar como eu vivi E tudo o que aconteceu comigo </p>Viver é melhor que sonhar E eu sei que o amor É uma coisa boa Mas também sei Que qualquer canto É menor do que a vida De qualquer pessoa</p>Por isso cuidado, meu bem Há perigo na esquina Eles venceram e o sinal Está fechado pra nós Que somos jovens</p>Para abraçar seu irmão E beijar sua menina na rua É que se fez o seu braço O seu lábio e a sua voz</p>Você me pergunta Pela minha paixão Digo que estou encantada Como uma nova invenção Eu vou ficar nesta cidade Não vou voltar pro sertão Pois vejo vir vindo no vento Cheiro de nova estação Eu sei de tudo na ferida viva Do meu coração</p>Já faz tempo Eu vi você na rua Cabelo ao vento Gente jovem reunida Na parede da memória Essa lembrança É o quadro que dói mais</p>Minha dor é perceber Que apesar de termos Feito tudo o que fizemo

# teste

url = 'https://www.letras.mus.br/belchior/600307/'

try:
        req = requests.get(url)
except requests.exceptions.RequestException as e:  # This is the correct syntax
        print(e)        
content = req.content
soup = BeautifulSoup(content, 'html.parser')
    
#compositor
#<div class="letra-info_comp">
html_comp = soup.find_all("div", {"class": "letra-info_comp"})
html_comp = str(html_comp)
compositor = html_comp.split('Composição: ')[1]
compositor = compositor.split('<a')[0]
compositor = compositor[0:-2].strip()
primeiro_comp = compositor.split(' / ')[0]  

a = 'belchior'
comp = a.replace('-',' ')
comp

c = primeiro_comp.lower()
c = remover_acentos(c)
c

# teste

url = 'https://www.letras.mus.br/ze-ramalho/49364/'

try:
        req = requests.get(url)
except requests.exceptions.RequestException as e:  # This is the correct syntax
        print(e)        
content = req.content
soup = BeautifulSoup(content, 'html.parser')
    
#letra da música
html = soup.find_all("div", {"class": "cnt-letra p402_premium"})
html = str(html)
estrofes = html.split('<p>')
estrofes = estrofes[1:]

letra = ''
for estrofe in estrofes:
    estrofe = estrofe.lower()
    #estrofe = estrofe.replace('</p>',' ')
    estrofe = estrofe.replace('<br/>',' ')
    estrofe = estrofe.replace('</br>',' ')
    estrofe = estrofe.replace('<br>',' ')
    estrofe = estrofe.replace('</div>]','')
    letra += estrofe

letra = letra.strip()
letra

## Visitando cada página, extraindo e armazenando as informações

In [93]:
from unicodedata import normalize
def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

In [95]:
letras_col = []
mais_tocada_col = []
titulo_col = []
compositor = artista.replace('-',' ')
compositores = []

for url in links_musicas:
    t, c, pc, mt, l = parse_lyric(url)
    
    print(t)
    print(c)
    print(mt)
    print(l)
    print()
    
    c = c.lower()
    c = remover_acentos(c)
    pc = pc.lower()
    pc = remover_acentos(pc)
    

    
    if original_composer:
    #só adciona ao dataframe se o compositor estiver definido na página e for igual ao artista
    #algumas letras podem não ter compositor definido. pode-se remover essa condição, nesses casos
    #também testa se já existe alguma letra com o mesmo título no dataset
    #se houver mais de um compositor, testará se o primeiro é igual ao artista buscado
        if (c == compositor or pc == compositor) and (t not in titulo_col):
            letras_col.append(l)
            mais_tocada_col.append(mt)
            titulo_col.append(t)
            compositores.append(c)
            
    else:
        letras_col.append(l)
        mais_tocada_col.append(mt)
        titulo_col.append(t)
        compositores.append(c)
    
   
    
    

1,2,3 Balançou
Alcyr Pires Vermelho
0
1,2,3, balançou 1,2,3, o meu samba eu quero assim Requebra e vai por mim </p>1,2,3, vai não vai 1,2,3, ouve o prato a marcar  O nosso balançar</p>Faz um passo todo figurado E segue o rebolado A onda do balanço me enrolou</p>Eu não posso parar Vejo o tempo correr sem me cansar Você pode notar</p>Batam palmas para quem gostou Do meu samba que a ninguém negou</p>De todo o coração Sensação Enquanto balançou</p>

20 Anos Blues
Sueli Costa / Vitor Martins
0
Ontem de manhã quando acordei Olhei a vida e me espantei Eu tenho mais de vinte anos E eu tenho mais de mil perguntas sem respostas Estou ligada num futuro blue </p>Os meus pais nas minhas costas As raízes na marquise Eu tenho mais de vinte muros O sangue jorra pelos furos, pelas veias de um jornal Eu não te quero, eu te quero mal</p>Essa calma que inventei, bem sei Custou as contas que contei Eu tenho mais de vinte anos E eu quero as cores e os colírios, meus delírios Estou ligada num futuro blue</p>

Acender as Velas
Zé Kéti
0
Acender as velas Já é profissão Quando não tem samba Tem desilusão É mais um coração Que deixa de bater Um anjo vai pro céu Deus me perdoe Mas vou dizer O doutor chegou tarde demais Porque no morro Não tem automóvel pra subir Não tem telefone pra chamar E não tem beleza pra se ver E a gente morre sem querer morrer </p>

Agnus Sei
Aldir Blanc / João Bosco
0
Faces sob o sol, os olhos na cruz Os heróis do bem prosseguem na brisa da manhã Vão levar ao reino dos minaretes a paz na ponta dos arietes A conversão para os infiéis </p>Para trás ficou a marca da cruz Na fumaça negra vinda na brisa da manhã Ah, como é difícil tornar-se herói Só quem tentou sabe como dói vencer Satã só com orações Ê andá pa Catarandá que Deus tudo vê Ê andá pa Catarandá que Deus tudo vê Ê anda, ê orar, ê mandar, ê matar, responderei não!</p>Dominus dominium juros além Todos esses anos agnus sei que sou também Mas ovelha negra me desgarrei, o meu pastor não sabe que eu sei Da arma oculta n

Amante À Moda Antiga
Não identificado
0


Amor a Natureza
Paulinho da Viola
0
Amor À Natureza Paulinho da Viola Composição: (Paulinho da Viola) </p>Relíquia do folclore nacional Jóia rara que apresento Nesta paisagem em que me vejo No centro da paixão e do tormento Sem nenhuma ilusão Neste cenário de tristeza Relembro momentos de real bravura Dos que lutaram com ardor Em nome do amor à natureza</p>Cinzentas nuvens de fumaça Umedecendo meus olhos De aflição e de cansaço Imensos blocos de concreto Ocupando todos os espaços Daquela que já foi a mais bela cidade Que o mundo inteiro consagrou Com suas praias tão lindas Tão cheias de graça, de sonho e de amor</p>Flutua no ar o desprezo Desconsiderando a razão Que o homem não sabe se vai encontrar Um jeito de dar um jeito na situação Uma semente atirada Num solo fértil não deve morrer É sempre uma nova esperança Que a gente alimenta de sobreviver</p>

Amor Até o Fim
Gilberto Gil
0
Amor não tem que se acabar Eu quero e sei que vou ficar Até o 

Aquele Abraço
Gilberto Gil
0
O Rio de Janeiro continua lindo O Rio de Janeiro continua sendo O Rio de janeiro, fevereiro e março Alô, alô, Realengo - aquele abraço! Alô, torcida do flamengo - aquele abraço! Chacrinha continua balançando a pança E buzinando a moça e comandando a massa E continua dando as ordens no terreiro Alô, alô, seu Chacrinha - velho guerreiro Alô, alô, Terezinha - Rio de janeiro Alô, alô, seu Chacrinha - velho palhaço Alô, alô, Terezinha - aquele abraço! Alô, moça da favela - aquele abraço! Todo mundo da Portela - aquele abraço! Todo mês de fevereiro - aquele passo! Alô, Banda de Ipanema - aquele abraço! Meu carinho pelo mundo eu mesmo traço A Bahia já me deu régua e compasso Quem sabe de mim sou eu - aquele abraço! Pra você que meu esqueceu aquele abraço! Alô, Rio de janeiro - aquele abraço! Todo o povo brasileiro - aquele abraço! </p>

Aqui É o País do Futebol
Fernando Brant / Milton Nascimento
0
Brasil está vazio na tarde de domingo, né? Olha o sambão, aqui é o 

Aviso Aos Navegantes
Baden Powell / Paulo César Pinheiro
0
Esse ano vai sobrar um Quem falou já morreu Quem sabe dele sou eu A vida quem dá é Deus </p>Quem é malandro não dá Vidinha boa a ninguém Malandro traz no cantar A pinta que o canto tem Briga com quem sabe mais Não dá camisa a ninguém Olha, aqui você faz Aqui mesmo vai entrar bem</p>Esse ano vai sobrar um Quem falou já morreu Quem sabe dele sou eu A vida quem dá é Deus</p>Vai sobrar um Falou, morreu Quem sabe dele sou eu Vida quem dá é Deus</p>

Baby Face
Akst / Davis / Fred Jorge
0
Baby face Esse rostinho lindo Baby face Parece ingênuo, não é isso não </p>Baby face Que encontrei sorrindo E me deixou sonhando</p>Baby face Esse seu modo estranho de olhar pra mim Me faz acreditar, que você Sabe amar,  mentindo Baby Face</p>

Baila Comigo
Rita Lee
0
Se Deus quiser, um dia eu quero ser índio Viver pelado pintado de verde, num eterno domingo Ser um bicho-preguiça, espantar turista E tomar banho de sol, banho de sol, banho de sol, sol

...

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Só Tinha De Ser Com Você
Aloysio de Oliveira / Antonio Carlos Jobim
1
É Só eu sei Quanto amor Eu guardei Sem saber Que era só Pra você </p>É, só tinha de ser com você Havia de ser pra você Senão era mais uma dor Senão não seria o amor Aquele que a gente não vê O amor que chegou para dar O que ninguém deu pra você O amor que chegou para dar O que ninguém deu pra você</p>É, você que é feito de azul Me deixa morar nesse azul Me deixa encontrar minha paz Você que é bonito demais Se ao menos pudesse saber Que eu sempre fui só de você Você sempre foi só de mim</p>É, você que é feito de azul Me deixa morar nesse azul Me deixa encontrar minha paz Você que é bonito demais Se ao menos pudesse saber Que eu sempre fui só de você Você sempre foi só de mim Eu sempre fui só de você Você sempre foi só de mim Eu sempre fui só de você Você sempre foi só de mim Eu sempre fui só de você Você sempre foi só de mim</p>

Somewhere
Não identificado
0
There's a place for us A time and space for us Peace and qui

These Are The Songs
Tim Maia
0
These are the songs I want to sing These are the songs I want to play I will sing it every day These are the songs I want to sing and play </p>Essa é a canção que eu vou ouvir Essa é a canção que eu vou cantar Fala de você, meu bem E do nosso amor, também Sei que você vai gostar</p>

Tiro Ao Álvaro
Adoniram Barbosa / Oswaldo Molles
1
De tanto levar frechada do teu olhar Meu peito até parece sabe o quê? Tábua de tiro ao Álvaro Não tem mais onde furar </p>De tanto levar frechada do teu olhar Meu peito até parece sabe o quê? Tábua de tiro ao Álvaro Não tem mais onde furar</p>Teu olhar mata mais do que bala de carabina Que veneno estricnina Que peixeira de baiano Teu olhar mata mais que atropelamento de automóvel Mata mais que bala de revólver</p>De tanto levar frechada do teu olhar Meu peito até parece sabe o quê? Tábua de tiro ao Álvaro Não tem mais onde furar</p>De tanto levar frechada do teu olhar Meu peito até parece sabe o quê? Tábua de tiro ao Álvaro N

Valsa de Euridice
Vinícius de Moraes
0
Tantas vezes já partiste Que chego a desesperar Chorei tanto, estou tão triste Que já nem sei mais chorar </p>Oh, meu amado, não parta Não parta de mim Oh, uma partida que não tem fim Não há nada que conforte</p>A falta dos olhos teus Pensa que a saudade Pode matar-me Adeus</p>

Valsa Rancho
Chico Buarque / Francis Hime
0
Valsa, rancha Me faz esquecer Esperar Em mil lágrimas Mil lágrimas Mil lágrimas </p>Valsa, rancha Me faz responder Transbordar Em mil lágrimas Mil lágrimas Mil lágrimas</p>Mil abraços Mil fracassos Meu delírio Teus pedaços Teu calor Seja feito Teu desejo Seja um beijo Seja como for Quantos braços Mil regaços Mil e uma noites Faz uma outra vez Como se ainda fosse Como é doce Como você fez</p>Me valsa Me rancha Me faz Me deixa Que criança Que esperança Que prazer Que saudade Que maldade Piedade Que fartura Que loucura Que cruel tortura Nos carinhos teus Minha santa criatura Diz que jura Pela mãe de Deus</p>Valsa, rancha Me faz desf

Vou Comprar Um Coração
Paulo Tito / Romeu Nunes
0
Vou comprar um coração Para trocar por este meu Porque sem ilusão não sei viver E este meu coração cansou de amar Com um novo coração Vou repetir os erros meus Fazer o que já fiz Amar em vão, ser infeliz Mais uma vez Com um novo coração Vou repetir os erros meus Fazer o que já fiz Amar em vão, ser infeliz Mais uma vez </p>

Vou Deitar e Rolar
Baden Powell / Paulo César Pinheiro
0
Não venha querer se consolar Que agora não dá mais pé Nem nunca mais vai dar Também, quem mandou se levantar? Quem levantou pra sair Perde o lugar </p>E agora, cadê teu novo amor? Cadê, que ele nunca funcionou? Cadê, que ele nada resolveu?</p>Quaquaraquaquá, quem riu? Quaquaraquaquá, fui eu Quaquaraquaquá, quem riu? Quaquaraquaquá, fui eu</p>Ainda sou mais eu</p>Você já entrou na de voltar Agora fica na tua Que é melhor ficar Porque vai ser fogo me aturar Quem cai na chuva Só tem que se molhar</p>E agora cadê, cadê você? Cadê que eu não vejo mais, cadê? Pois é,

In [96]:
len(titulo_col)

386

## Montando um DataFrame

In [100]:
df = pd.DataFrame(data = {'titulo':titulo_col, 
                         'mais tocada': mais_tocada_col,
                         'letra': letras_col,
                         'compositor': compositores})

In [101]:
df

,titulo,mais tocada,letra,compositor
0,"1,2,3 Balançou",0,"1,2,3, balançou 1,2,3, o meu samba eu quero as...",alcyr pires vermelho
1,20 Anos Blues,0,Ontem de manhã quando acordei Olhei a vida e m...,sueli costa / vitor martins
2,A Banca do Distinto,0,"Não fala com pobre, não dá mão a preto Não car...",billy blanco
3,A Corujinha,0,"Corujinha, corujinha, Que peninha de você. Fic...",toquinho / vinicius de moraes
4,A Dama do Apocalipse,0,Branco por cima e o negro de um sorriso herói ...,nathan marques
...,...,...,...,...
381,Yesterday,0,Yesterday All my troubles seemed so far away N...,the beatles
382,Zambi,0,"É Zambi na noite Ei,Ei, é ZAmbi É Zambi, tue, ...",tetes raides / vinicius de moraes
383,Zazueira,0,Ela vem chegando E feliz vou esperando A esper...,jorge ben
384,Zé Não É João,0,"Pra que que eu fui telefonar, pra quê? Pra que...",ciro monteiro


# Exportando CSV

In [102]:
filename = artista + '_raw.csv'
filename

'elis-regina_raw.csv'

In [ ]:
df.to_csv(filename, index = False)